In [1]:
import pandas as pd
from darts import TimeSeries
from darts.models import TFTModel
from darts.dataprocessing.transformers import Scaler
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from darts.utils.likelihood_models import QuantileRegression
import sys
import json
sys.path.append('../Helper/')
from dataPreprocessing import get_untransformed_exog, TRAIN_DATA_PATH_1990S, TEST_DATA_PATH_1990S
from PyTorchModular import darts_optuna,save_model_hyper_params, train_valid_split_darts, load_prediction
from torch.nn import MSELoss
from torch.optim import AdamW
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
import numpy as np

In [2]:

data = pd.read_csv(TRAIN_DATA_PATH_1990S)
data= get_untransformed_exog(data)
df= data.copy()
df['observation_date'] = pd.to_datetime(df['observation_date'], format='%m/%Y')
df.set_index('observation_date', inplace=True)
df_exog= df.drop('fred_PCEPI',axis=1)

test_data = pd.read_csv(TEST_DATA_PATH_1990S)
test_data= get_untransformed_exog(test_data.copy())
test_data['observation_date'] = pd.to_datetime(test_data['observation_date'], format='%m/%Y')
test_data.set_index('observation_date', inplace=True)
test_exog= test_data.drop('fred_PCEPI',axis=1)
test_target= test_data['fred_PCEPI']

In [3]:
N_TRIALS=150

In [4]:
search_space = {
    'input_chunk_length': (int, 1, 36,{'step':1,'log':False}),
    'lstm_layers': (int, 1, 32,{'step':1,'log':False}),
    'num_attention_heads': (int, 1, 4,{'step':1,'log':False}),
    'hidden_size': (int, 1, 32,{'step':1,'log':False}),
    'dropout': (float, 0., 0.5,{'step':0.1,'log':False}),
    'optimizer_kwargs': {"lr": (float, 1e-5, 1e-1,{'step':None,'log':True})},
    'loss_fn':('categorical',["QuantileRegression", "MSE"])
}

In [5]:
def load_TFT_params(json_path):

    with open(json_path,'r') as f:
        loaded_params= json.load(f)

    if 'loss_fn' in loaded_params.keys():
        if loaded_params['loss_fn']=='MSE':
            loaded_params['loss_fn']=MSELoss()
            loaded_params['likelihood']=None
        else:
            loaded_params['loss_fn']=None
            loaded_params['likelihood']=QuantileRegression((0.25,0.5,0.75))

    loaded_params['optimizer_kwargs']={"lr":loaded_params["lr"]}
    del loaded_params["lr"]

    return loaded_params


def train_TFT(best_params_path, valid_size,horizon):
    
    best_params= load_TFT_params(best_params_path)
    target_scaler= Scaler()
    exog_scaler=Scaler()

    probabilistic= True if best_params['loss_fn'] is None else False

    train_target, valid_target, train_exog, valid_exog=train_valid_split_darts(df_exog.copy(),df['fred_PCEPI'].copy(),valid_size,best_params['input_chunk_length'])

    train_target= target_scaler.fit_transform(train_target)
    train_exog= exog_scaler.fit_transform(train_exog)

    valid_target= target_scaler.transform(valid_target)
    valid_exog= exog_scaler.transform(valid_exog)

    early_stopper = EarlyStopping(
                monitor="val_loss",
                patience=25,
                min_delta=1e-5,
                mode="min",
                verbose=True
                )
    best_params["save_checkpoints"]=True
    best_params["force_reset"]=True
    best_params["random_state"]=42
    best_params["output_chunk_length"]=horizon
    best_params['optimizer_cls']=AdamW
    best_params['add_encoders']={
        "cyclic": {"future": ["month", "quarter"]}
        }

    best_params["pl_trainer_kwargs"]={
                "accelerator":'auto',
                "callbacks": [early_stopper]
            }

    model = TFTModel(**best_params, model_name= f'TFT_horizon_{horizon}')

    model.fit(train_target,
            past_covariates=train_exog,
            val_series=valid_target,
            val_past_covariates=valid_exog,
            epochs=10000)

    model.load_from_checkpoint(model_name=f'TFT_horizon_{horizon}',best=True)

    return model, target_scaler, exog_scaler, best_params['input_chunk_length'] ,probabilistic

def infer_darts(model,train_exog_df, train_target_series, test_exog_df, test_target_series,exog_scaler,target_scaler, input_size,horizon, probabilistic=False):
    
    combined_exog_df= pd.concat((train_exog_df,test_exog_df),axis=0)
    combined_target_series= pd.concat((train_target_series,test_target_series),axis=0)

    preds_list=np.array([])

    for i in range(0,12,horizon):
        x,y=load_prediction(input_size,combined_exog_df,combined_target_series,exog_scaler,target_scaler,test_target.index[i])
        if probabilistic:
            pred=target_scaler.inverse_transform(model.predict(n=horizon,series=y,past_covariates=x,verbose=False, num_samples=1000)).values().flatten()
        else:
            pred=target_scaler.inverse_transform(model.predict(n=horizon,series=y,past_covariates=x,verbose=False)).values().flatten()

        print(test_target.index[i:i+horizon])
        print(pred)
        preds_list= np.append(preds_list,pred)
    
    return preds_list

## Horizon = 1:

In [ ]:
HORIZON=1
VALID_SIZE=3
invariates= {
    'output_chunk_length':HORIZON,
    'optimizer_cls':AdamW,
    'add_encoders':{
        "cyclic": {"future": ["month", "quarter"]}
        }
}

best_params=darts_optuna(TFTModel,'TFT',search_space,invariates,df['fred_PCEPI'].copy(),df_exog.copy(),VALID_SIZE,HORIZON,n_trials=N_TRIALS,patience=5,tol=1e-4,verbose=True)
save_model_hyper_params('best_params_tft_horizon_1.json',best_params)

In [6]:
model_horizon_1, target_scaler_horizon_1, exog_scaler_horizon_1,input_size_horizon_1,probabilistic_horizon_1= train_TFT('best_params_tft_horizon_1.json',valid_size=3,horizon=1)
preds_horizon_1=infer_darts(model_horizon_1, df_exog,df['fred_PCEPI'],
                            test_exog,test_target,exog_scaler=exog_scaler_horizon_1,
                            target_scaler=target_scaler_horizon_1,input_size=input_size_horizon_1,
                            horizon=1, probabilistic=probabilistic_horizon_1)

np.save('TFT_horizon_1.npy',preds_horizon_1)

2025-04-21 14:55:16,717 - INFO - Train dataset contains 378 samples.
2025-04-21 14:55:17,051 - INFO - Time series values are 64-bits; casting model to float64.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name                              | Type                             | Params | Mode 
------------------------------------------------------------------------------------------------
0  | criterion                         | MSELoss                          | 0      | train
1  | train_criterion                   | MSELoss                          | 0      | train
2  | val_criterion                     | MSELoss                          | 0      | train
3  | train_metrics                     | MetricCollection                 | 0      | train
4  | val_metrics                       | MetricCollection                 | 0      | train
5  | input_embeddings                 

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Metric val_loss improved. New best score: 0.453


...

DatetimeIndex(['2024-09-01'], dtype='datetime64[ns]', name='observation_date', freq=None)
[120.87731274]


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


DatetimeIndex(['2024-10-01'], dtype='datetime64[ns]', name='observation_date', freq=None)
[120.89346668]


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


DatetimeIndex(['2024-11-01'], dtype='datetime64[ns]', name='observation_date', freq=None)
[120.71575519]
DatetimeIndex(['2024-12-01'], dtype='datetime64[ns]', name='observation_date', freq=None)
[120.8378769]


## Horizon = 3:

In [ ]:
HORIZON=3
VALID_SIZE=3
invariates= {
    'output_chunk_length':HORIZON,
    'optimizer_cls':AdamW,
    'add_encoders':{
        "cyclic": {"future": ["month", "quarter"]}
        }
}

best_params=darts_optuna(TFTModel,'TFT',search_space,invariates,df['fred_PCEPI'].copy(),df_exog.copy(),VALID_SIZE,HORIZON,n_trials=N_TRIALS,patience=5,tol=1e-4,verbose=True)
save_model_hyper_params('best_params_tft_horizon_3.json',best_params)

In [8]:
model_horizon_3, target_scaler_horizon_3, exog_scaler_horizon_3,input_size_horizon_3,probabilistic_horizon_3= train_TFT('best_params_tft_horizon_3.json',valid_size=3,horizon=3)
preds_horizon_3=infer_darts(model_horizon_3, df_exog,df['fred_PCEPI'],test_exog,test_target,exog_scaler=exog_scaler_horizon_3,
                            target_scaler=target_scaler_horizon_3,input_size=input_size_horizon_3,horizon=3,
                            probabilistic=probabilistic_horizon_3)

np.save('TFT_horizon_3.npy',preds_horizon_3)

2025-04-21 15:03:31,937 - INFO - Train dataset contains 372 samples.
2025-04-21 15:03:32,159 - INFO - Time series values are 64-bits; casting model to float64.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name                              | Type                             | Params | Mode 
------------------------------------------------------------------------------------------------
0  | train_metrics                     | MetricCollection                 | 0      | train
1  | val_metrics                       | MetricCollection                 | 0      | train
2  | input_embeddings                  | _MultiEmbedding                  | 0      | train
3  | static_covariates_vsn             | _VariableSelectionNetwork        | 0      | train
4  | encoder_vsn                       | _VariableSelectionNetwork        | 147 K  | train
5  | decoder_vsn                      

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Metric val_loss improved. New best score: 0.708


...

DatetimeIndex(['2024-01-01', '2024-02-01', '2024-03-01'], dtype='datetime64[ns]', name='observation_date', freq=None)
[121.92669092 121.93596094 120.00929923]


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


DatetimeIndex(['2024-04-01', '2024-05-01', '2024-06-01'], dtype='datetime64[ns]', name='observation_date', freq=None)
[122.08536339 120.23789367 123.62485093]


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


DatetimeIndex(['2024-07-01', '2024-08-01', '2024-09-01'], dtype='datetime64[ns]', name='observation_date', freq=None)
[120.6584226  122.83966029 122.22171285]
DatetimeIndex(['2024-10-01', '2024-11-01', '2024-12-01'], dtype='datetime64[ns]', name='observation_date', freq=None)
[122.5320386  121.52521784 121.72586596]


## Horizon = 6:

In [ ]:
HORIZON=6
VALID_SIZE=6
invariates= {
    'output_chunk_length':HORIZON,
    'optimizer_cls':AdamW,
    'add_encoders':{
        "cyclic": {"future": ["month", "quarter"]}
        }
}

best_params=darts_optuna(TFTModel,'TFT',search_space,invariates,df['fred_PCEPI'].copy(),df_exog.copy(),VALID_SIZE,HORIZON,n_trials=N_TRIALS,patience=5,tol=1e-4,verbose=True)
save_model_hyper_params('best_params_tft_horizon_6.json',best_params)

In [10]:
model_horizon_6, target_scaler_horizon_6, exog_scaler_horizon_6,input_size_horizon_6,probabilistic_horizon_6= train_TFT('best_params_tft_horizon_6.json',valid_size=6,horizon=6)
preds_horizon_6=infer_darts(model_horizon_6, df_exog,df['fred_PCEPI'],
                            test_exog,test_target,exog_scaler=exog_scaler_horizon_6,
                            target_scaler=target_scaler_horizon_6,input_size=input_size_horizon_6,
                            horizon=6, probabilistic=probabilistic_horizon_6)

np.save('TFT_horizon_6.npy',preds_horizon_6)

2025-04-21 15:28:25,994 - INFO - Train dataset contains 365 samples.
2025-04-21 15:28:26,437 - INFO - Time series values are 64-bits; casting model to float64.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name                              | Type                             | Params | Mode 
------------------------------------------------------------------------------------------------
0  | criterion                         | MSELoss                          | 0      | train
1  | train_criterion                   | MSELoss                          | 0      | train
2  | val_criterion                     | MSELoss                          | 0      | train
3  | train_metrics                     | MetricCollection                 | 0      | train
4  | val_metrics                       | MetricCollection                 | 0      | train
5  | input_embeddings                 

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Metric val_loss improved. New best score: 0.207


...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Monitored metric val_loss did not improve in the last 25 records. Best score: 0.000. Signaling Trainer to stop.
2025-04-21 15:36:56,554 - INFO - loading best-epoch=34-val_loss=0.00.ckpt
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


DatetimeIndex(['2024-01-01', '2024-02-01', '2024-03-01', '2024-04-01',
               '2024-05-01', '2024-06-01'],
              dtype='datetime64[ns]', name='observation_date', freq=None)
[119.63394115 119.28177335 118.8666288  118.83590973 119.04168772
 119.94777519]
DatetimeIndex(['2024-07-01', '2024-08-01', '2024-09-01', '2024-10-01',
               '2024-11-01', '2024-12-01'],
              dtype='datetime64[ns]', name='observation_date', freq=None)
[119.62716442 119.48649036 119.20373564 120.0917914  120.07498062
 120.17600453]


## Horizon = 12:

In [ ]:
HORIZON=12
VALID_SIZE=12
invariates= {
    'output_chunk_length':HORIZON,
    'optimizer_cls':AdamW,
    'add_encoders':{
        "cyclic": {"future": ["month", "quarter"]}
        }
}

best_params=darts_optuna(TFTModel,'TFT',search_space,invariates,df['fred_PCEPI'].copy(),df_exog.copy(),VALID_SIZE,HORIZON,n_trials=N_TRIALS,patience=5,tol=1e-4,verbose=True)
save_model_hyper_params('best_params_tft_horizon_12.json',best_params)

In [11]:
model_horizon_12, target_scaler_horizon_12, exog_scaler_horizon_12,input_size_horizon_12,probabilistic_horizon_12= train_TFT('best_params_tft_horizon_12.json',valid_size=12,horizon=12)
preds_horizon_12=infer_darts(model_horizon_12, df_exog,df['fred_PCEPI'],
                            test_exog,test_target,exog_scaler=exog_scaler_horizon_12,
                            target_scaler=target_scaler_horizon_12,input_size=input_size_horizon_12,
                            horizon=12, probabilistic=probabilistic_horizon_12)

np.save('TFT_horizon_12.npy',preds_horizon_12)

2025-04-21 15:37:26,481 - INFO - Train dataset contains 364 samples.
2025-04-21 15:37:26,639 - INFO - Time series values are 64-bits; casting model to float64.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name                              | Type                             | Params | Mode 
------------------------------------------------------------------------------------------------
0  | criterion                         | MSELoss                          | 0      | train
1  | train_criterion                   | MSELoss                          | 0      | train
2  | val_criterion                     | MSELoss                          | 0      | train
3  | train_metrics                     | MetricCollection                 | 0      | train
4  | val_metrics                       | MetricCollection                 | 0      | train
5  | input_embeddings                 

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Metric val_loss improved. New best score: 0.148


...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Monitored metric val_loss did not improve in the last 25 records. Best score: 0.000. Signaling Trainer to stop.
2025-04-21 15:46:42,477 - INFO - loading best-epoch=44-val_loss=0.00.ckpt
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


DatetimeIndex(['2024-01-01', '2024-02-01', '2024-03-01', '2024-04-01',
               '2024-05-01', '2024-06-01', '2024-07-01', '2024-08-01',
               '2024-09-01', '2024-10-01', '2024-11-01', '2024-12-01'],
              dtype='datetime64[ns]', name='observation_date', freq=None)
[115.90663122 116.18828234 117.10502668 118.32705551 119.26170811
 120.25691077 120.76050368 121.18323754 121.44794927 121.94273968
 121.50802858 120.95406353]
